# QC Tutorial

---

This interactive tutorial will show you a couple ways to quality control data using LROSE using RadxPid/RadxRate and RadxQc. RadxPid/RadxRate allows users to censor non-weather echoes using the National Center for Atmospheric Research (NCAR) particle identification (PID) algorithm and RadxQc. 

RadxPid is one of three 'Echo' applications in LROSE. A visual comparison of RadxPid with RadxKdp and RadxRate and their parameter files is shown below. Each application has its own main parameter file that defines variable names and specifies the paths to the parameter file for each relevant sub-process. For example, the main RadxPid parameter file links to the specific differential phase (KDP) and PID parameter files (the PID parameter file links to the PID thresholds file).

<div>
<img src="../images/radx_echo_description.png" width="600"/>
</div>

---

*Note: this tutorial shows two examples of how to quality control radar data. Other workflows exist!*

---

## Tutorial Overview
### 1. Setup

#### Directory organization

The structure of the QC tutorial on JupyterHub is shown in the diagram below. The parent or base directory is "lrose-hub" and contains all of the notebooks, parameter files, and data for the workshop.

<div>
<img src="../images/qc_structure.png" width="500"/>
</div>

#### Raw data and parameter files

CfRadial data file that will be provided:
* cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc

Parameter files (included in this tutorial):

*Note: The parameter files have already been modified to run straight out of the box.* 
* RadxPid main params
* RadxRate main params
* RadxPid Kdp_specific params
* RadxPid Pid_specific params
* Pid Thresholds params (S-band, simultaneous transmit)
* RadxQc params

### 2. Run RadxPid/RadxRate - censor non-weather gates

* Run PID algorithm to identify gates which do not have weather
    * RadxPid/RadxRate

### 3. Run RadxQc - censor based on thresholds

* Censor gates where RHOHV and NCP don't meet thresholds
    * RadxQc

### 4. Plot PID and Rate

* Visualize results of RadxPid/RadxRate and RadxQc analysis using Py-ART

---

For this tutorial, the parameter files have already been created and populated with the appropriate parameters. However, the default parameter files can be saved using the following commands.

In [ ]:
# # create original parameter files
# !RadxPid -print_params > /home/jovyan/ams2023/params/user/RadxPid_main_params_user
# !RadxRate -print_params > /home/jovyan/ams2023/params/user/RadxRate_main_params_user
# !RadxPid -print_params_kdp > /home/jovyan/ams2023/params/user/RadxPid_Kdp_params_user
# !RadxPid -print_params_pid > /home/jovyan/ams2023/params/user/RadxPid_Pid_params_user
# !RadxQc -print_params > /home/jovyan/ams2023/params/user/RadxQc_params_user

# to create new parameter files that use parameters from another file
# !RadxPid -params /path/to/params -print_params > /home/jovyan/ams2023/params/user/RadxPid_main_params_user

# for command line options
# !RadxPid -h


# 1. Setup
## Environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. At the end we will plot the output using Py-ART.

In [ ]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

Next, we set up the directory structure to simplify our commands. If you are running this notebook at the LROSE workshop on JupyterHub, these paths go to the parent directory containing all the workshop resources and the LROSE binaries. 

**If you have downloaded this notebook, please modify BASE_DIR to work on your personal machine.**

* BASE_DIR: the base directory containing the directories for the notebooks, data, parameter files
* RADAR_NAME: the name of the radar used in this tutorial

In [ ]:
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['RAW_DIR'] = '/home/jovyan/share/raw'
os.environ['RADAR_NAME'] = 'CHILL_S'
base_dir = os.environ['BASE_DIR']
radar_name = os.environ['RADAR_NAME']
!echo "Base directory: "$BASE_DIR
!echo "Radar name: "$RADAR_NAME

## Set up directories

We need to set up the required data directories. The radar data is provided on the JupyterHub. We delete any existing files and directories specific to this tutorial to ensure we're starting with clean directories and files.

In [ ]:
## make subdirectory within data for the raw data
!rm -rf ${BASE_DIR}/data/qc/raw
!mkdir -p ${BASE_DIR}/data/qc/raw


We can examine the raw radar file using RadxPrint. RadxPrint can read CfRadial and other raw radar formats supported by LROSE that can later be converted by RadxConvert. It will also provide information on radar variables and sweep information. For example, we can look at the variables using the following command, piping the output into the head command.

In [ ]:
# print out the first 50 lines of RadxPrint output
!RadxPrint -f ${RAW_DIR}/qc/*.nc | head -50


# 2.  Run RadxPid - censor non-weather gates

## PID Algorithm

The NCAR PID algorithm uses polarimetric radar data to classify radar gates into 17 distinct categories, 3 of which are considered non-weather categories: insects, second-trip echoes, and clutter. RadxPid and RadxRate have the capability to censor gates that are not weather in the written output files. Thus, we will run the PID algorithm to censor the data. 

Since censoring does not care too much about the sounding quality (used for melting level identification), this tutorial will use a rough sounding from Denver 3 hours before this radar file, which will be included in the PID thresholds file. 

Either RadxPid or RadxRate can be used.

## RadxPid/RadxRate params

**RadxRate main parameters (line numbers given for RadxRate)**
| Parameter | Line # | Description |
| --- | --- | --- |
|input_dir| 98 |Path to input data, if not already specified on the command line.|
|SNR_available| 160 |Specify if SNR is available.|
|SNR_field_name| 170 |SNR variable name.|
|DBZ_field_name| 192 |DBZ variable name.|
|ZDR_field_name| 200 |ZDR variable name.|
|PHIDP_field_name| 208 |PHIDP variable name.|
|RHOHV_field_name| 216 |RHOHV variable name.|
|LDR_available| 224 |Specify if LDR is available.|
|LDR_field_name| 232 |LDR variable name.|
|kdp_params_file_path| 270 |Path to Kdp-specific parameter file.|
|pid_params_file_path| 287 |Path to PID-specific parameter file.|
|PID_use_attenuation_corrected_fields| 311 |Specify whether to use attenuation-corrected DBZ and ZDR in PID.|
|output_dir| 757 |Path where output files are written, if not already specified on command line with -outdir.|
    
**RadxPid/RadxRate Kdp_specific parameters**
| Parameter | Line # | Description |
| --- | --- | --- |
|KDP_fir_filter_len| 63 |Filter length used for KDP calculation.|
|KDP_psob_method| 108 |Specify method to remove phase shift on backscatter.|
          
**RadxPid/RadxRate Pid_specific parameters**
| Parameter | Line # | Description |
| --- | --- | --- |
|PID_thresholds_file_path| 25 |Path to fuzzy logic PID thresholds file.|
|PID_use_soundings_from_spdb| 259 |Specify whether soundings are in Spdb format, otherwise sounding found in fuzzy logic file.|
|PID_sounding_spdb_url| 267 |Path to Spdb soundings.|
|PID_sounding_location_name| 289 |Name of sounding location.|

**PID thresholds file**
| Parameter | Line # | Description |
| --- | --- | --- |
|Tpf| 57 |If not using Spdb soundings, a single sounding is drawn from this variable (altitude (km), temperature (C)).|
    
****

Here, *-params* provides the link to the main RadxRate parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where RadxRate should write the final files.

In [ ]:
!rm -rf ${BASE_DIR}/data/qc/radxpid
!mkdir -p ${BASE_DIR}/data/qc/radxpid

# run RadxPid or RadxRate
!RadxPid -params ${BASE_DIR}/params/qc/RadxPid_main_params -f ${RAW_DIR}/qc/*.nc -outdir ${BASE_DIR}/data/qc/radxpid
#!RadxRate -params ${BASE_DIR}/params/qc/RadxRate_main_params -f ${RAW_DIR}/qc/*.nc -outdir ${BASE_DIR}/data/qc/radxrate


# 3. Run RadxQc - censor based on thresholds

## RadxQc algorithm
RadxQc includes a few quality control algorithms, including KDP calculation, the PID algorithm, removing RLAN interference, and censoring based on variable thresholds. We will censor based on the values of RHOHV and NCP in this tutorial. In this example, we will only retain gates where RHOHV values range between 0.8 - 1.1 and NCP values range between 0.4 - 1.1. In addition, to remove short runs of noise, we will remove gates where the run of uncensored gates totals less than 5 gates. We will write DBZ, ZDR, PHIDP, RHOHV, and NCP to the output file, where only DBZ, PHIDP, and ZDR have been censored.

## RadxQc params

| Parameter | Line # | Description |
| --- | --- | --- |
|input_dir| 118 |Path to input data, if not specified on the command line.|
|DBZ_field_name| 378 |DBZ variable name.|
|VEL_field_name| 388 |VEL variable name.|
|WIDTH_available| 399 |Specify if WIDTH is available.|
|WIDTH_field_name| 407 |WIDTH variable name.|
|NCP_available| 418 |Specify if NCP is available, in this example FALSE.|
|NCP_field_name| 426 |NCP variable name.|
|SNR_available| 437 |Specify if SNR is available, in this example FALSE.|
|SNR_field_name| 447 |SNR variable name.|
|ZDR_available| 469 |Specify if ZDR is available.|
|ZDR_field_name| 477 |ZDR variable name.|
|LDR_available| 485 |Specify if LDR is available, in this example FALSE.|
|LDR_field_name| 495 |LDR variable name.|
|PHIDP_available| 503 |Specify if PHIDP is available.|
|PHIDP_field_name| 511 |PHIDP variable name.|
|RHOHV_available| 519 |Specify if RHOHV is available.|
|RHOHV_field_name| 527 |RHOHV variable name.|
|compute_pid| 1334 |Tell RadxQc whether to run the PID algorithm for censoring, in this example FALSE.|
|censoring_input_fields| 1877 |Specify variables, variable thresholds, and combination method used to censor data.|
|input_field_censoring_min_valid_run| 1906 |Minimum valid run of non-censored gates.|
|censored_output_fields| 1946 |Specify the fields to be copied from the input files, censored, and written to the output files.|

****

Here, *-params* provides the link to the main RadxQc parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where RadxQc should write the final files.

In [ ]:
!rm -rf ${BASE_DIR}/data/qc/radxqc
!mkdir -p ${BASE_DIR}/data/qc/radxqc

# run RadxQc
!RadxQc -params ${BASE_DIR}/params/qc/RadxQc_params -f ${RAW_DIR}/qc/*.nc -outdir ${BASE_DIR}/data/qc/radxqc


# 4. Plot censored CHILL output

## Censoring using the PID

To visualize the output in the notebook, we can use Py-ART. HawkEye is also great for visualizing data - a general parameter file can be found in the echo parameter files directory.

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/qc/radxpid/20210815/"
file = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
rate_kmhx = pyart.io.read_cfradial(inDir+file)
rate_kmhx.info('compact')


We can create a colormap for visualizing PID.

In [ ]:
pidmap = np.array([[0.12156862745098039, 0.46666666666666667, 0.70588235294117652, 1.0],
              [0.68235294117647061, 0.7803921568627451, 0.90980392156862744, 1.0],
              [0.59607843137254901, 0.87450980392156863, 0.54117647058823526, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],
              [1.0, 0.49803921568627452, 0.054901960784313725, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [0.32156862745098042, 0.32941176470588235, 0.63921568627450975, 1.0],
              [0.859375, 0.859375, 0.859375, 1.0],
              [0.66015625, 0.66015625, 0.66015625, 1.0],
              [0.41015625, 0.41015625, 0.41015625, 1.0],
              [0.0, 0.0, 0.0, 1.0],],'f')
my_cmap2 = colors.ListedColormap(pidmap, name='ncar_pid')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('ZDR', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


## Censoring using the variable thresholds

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/qc/radxqc/20210815/"
file = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
qc_chill = pyart.io.read_cfradial(inDir+file)
qc_chill.info('compact')

In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(qc_chill)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('ZDR', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR (dB)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# RHOHV

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('RHOHV', 0, vmin=0.8, vmax=1.05,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RHOHV")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCP

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('NCP', 0, vmin=0.4, vmax = 1.1,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="NCP",
    mask_outside=True)
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

figRate.tight_layout()

plt.show()


# 5. Bonus fun - run RadxQc and then RadxPid/RadxRate!

User preferences with respect to noise, speckling, etc. will vary and often multiple processing steps are involved. Pre-processing using Py-ART or CSU RadarTools could be fed into RadxPid/RadxRate.

In [ ]:
!rm -rf ${BASE_DIR}/data/qc/radxqc_then_pid
!mkdir -p ${BASE_DIR}/data/qc/radxqc_then_pid

# run RadxPid/RadxRate using RadxQc output
!RadxPid -params ${BASE_DIR}/params/qc/RadxPid_main_params -f ${BASE_DIR}/data/qc/radxqc/20210815/*.nc -outdir ${BASE_DIR}/data/qc/radxqc_then_pid
#!RadxRate -params ${BASE_DIR}/params/qc/RadxRate_main_params -f ${BASE_DIR}/data/qc/radxqc/20210815/*.nc -outdir ${BASE_DIR}/data/qc/radxqc_then_pid

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/qc/radxqc_then_pid/20210815/"
file = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
pid_then_qc = pyart.io.read_cfradial(inDir+file)
pid_then_qc.info('compact')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(pid_then_qc)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('ZDR', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()
